<a href="https://colab.research.google.com/github/ShahistaAfreen/DL_DA6401_A3/blob/main/Without_Attention_Sweep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import package

In [ ]:
!pip install wandb
!pip install xtarfile

  Preparing metadata (setup.py) ... done
  Created wheel for xtarfile: filename=xtarfile-0.2.1-py3-none-any.whl size=8056 sha256=ea57fa9ed213e42a0f0f008d20ca19f453ef3726bc3ad78a901d660303001c6f
  Stored in directory: /root/.cache/pip/wheels/73/de/d6/def0eebfc3d5adb3a866d1bd9ae45649e07e6cffb284314a00
Successfully built xtarfile


In [ ]:
START_TOKEN="\t"
END_TOKEN="\n"

In [ ]:
import os
import random
import time
import wandb
import re, string
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from collections import Counter
import os
from os.path import exists
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import numpy as np
import keras
import tensorflow as tf
import csv

from tensorflow.keras import layers
import tensorflow.keras.backend as K




# **DataPreprocessing**

In [ ]:
import tensorflow as tf
import random
import numpy as np
import os

from os.path import exists
import xtarfile as tarfile
import pandas as pd
import keras
START_TOKEN="0"
END_TOKEN="1"

"""Download dataset if not exists"""
def downloadDataSet():
   cwd = os.getcwd()

   file_exists = exists('./dakshina_dataset_v1.0.tar')
   if(file_exists==False):
     print('downloading....')
     os.system('curl -SL https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar > dakshina_dataset_v1.0.tar')
     print('download Complete')
   extract_exists = exists('./dakshina_dataset_v1.0/')
   if(extract_exists==False):
     print('Extracting..')
     with tarfile.open('dakshina_dataset_v1.0.tar', 'r') as archive:
         archive.extractall()
     print('Complete')
   print('You are all set')
def get_files(language):

  train_dir='./dakshina_dataset_v1.0/'+language+'/lexicons/'+language+'.translit.sampled.train.tsv'
  val_dir='./dakshina_dataset_v1.0/'+language+'/lexicons/'+language+'.translit.sampled.dev.tsv'
  test_dir='./dakshina_dataset_v1.0/'+language+'/lexicons/'+language+'.translit.sampled.test.tsv'

  return train_dir, val_dir, test_dir
"""Generate Tokens"""
def tokenize(lang,tokenizer=None):
    """ Uses tf.keras tokenizer to tokenize the data/words into characters
    """
    if(tokenizer==None):
        tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
        tokenizer.fit_on_texts(lang)
        lang_tensor = tokenizer.texts_to_sequences(lang)
        lang_tensor = tf.keras.preprocessing.sequence.pad_sequences(lang_tensor,
                                                            padding='post')
    else:

        lang_tensor = tokenizer.texts_to_sequences(lang)
        lang_tensor = tf.keras.preprocessing.sequence.pad_sequences(lang_tensor,
                                                        padding='post')

    return lang_tensor, tokenizer

"""Preprocessing the data"""
def preprocess_data(fpath,ip_tokenizer=None, tgt_tokenizer=None):

    #Read data from files
    df = pd.read_csv(fpath, sep="\t", header=None)

    #Add start and end token
    df[0] = df[0].apply( lambda x:START_TOKEN+x+END_TOKEN)
    ip_tensor, ip_tokenizer = tokenize(df[1].astype(str).tolist(), tokenizer=ip_tokenizer)

    tgt_tensor, tgt_tokenizer = tokenize(df[0].astype(str).tolist(), tokenizer=tgt_tokenizer)

    dataset = tf.data.Dataset.from_tensor_slices((ip_tensor, tgt_tensor))
    dataset = dataset.shuffle(len(dataset))

    return dataset, ip_tokenizer, tgt_tokenizer





# **Bahdanau Attention**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow.keras.backend as K

"""
Bahdanau Attention
"""
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, enc_state, enc_out):

    enc_state = tf.concat(enc_state, 1)
    enc_state = tf.expand_dims(enc_state, 1)

    score = self.V(tf.nn.tanh(self.W1(enc_state) + self.W2(enc_out)))

    attention_weights = tf.nn.softmax(score, axis=1)

    context_vector = attention_weights * enc_out
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights




In [ ]:
!pwd

/content


In [ ]:
class Parameters():
  def  __init__(self,  language='te',encoder_layers=1,decoder_layers=1,embedding_dim=128,\
                layer_type='lstm', units=128, dropout=0.5, attention=False,attention_type="Luong",batch_size=128,\
                apply_beam_search=False,apply_teacher_forcing=False,teacher_forcing_ratio=1,\
                 save_outputs=None,epochs=5,wandb=None,beamWidth=5,restoreBestModel=True,\
                 patience=2,encoder_vocab_size=64,decoder_vocab_size=64):
        self.language = language
        self.embedding_dim = embedding_dim
        self.encoder_layers=encoder_layers
        self.decoder_layers=decoder_layers
        self.layer_type = layer_type
        self.units = units
        self.dropout = dropout
        self.attention = attention
        self.stats = []
        self.wandb=wandb
        self.epochs=epochs
        self.batch_size = 128
        self.apply_beam_search = apply_beam_search
        self.batch_size = batch_size
        self.apply_teacher_forcing=apply_teacher_forcing
        self.save_outputs=save_outputs
        self.restoreBestModel=restoreBestModel
        self.attention_type=attention_type
        self.patience=patience
        self.encoder_vocab_size=encoder_vocab_size
        self.decoder_vocab_size=decoder_vocab_size
        self.teacher_forcing_ratio=teacher_forcing_ratio

# **Encoder**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Assumed helper function
def get_layer(layer_type, units, dropout, return_sequences, return_state):
    if layer_type.lower() == "lstm":
        return tf.keras.layers.LSTM(units,
                                    return_sequences=return_sequences,
                                    return_state=return_state,
                                    dropout=dropout)
    elif layer_type.lower() == "gru":
        return tf.keras.layers.GRU(units,
                                   return_sequences=return_sequences,
                                   return_state=return_state,
                                   dropout=dropout)
    elif layer_type.lower() == "rnn":
        return tf.keras.layers.SimpleRNN(units,
                                   return_sequences=return_sequences,
                                   return_state=return_state,
                                   dropout=dropout)
    else:
        raise ValueError(f"Unsupported layer type: {layer_type}")

class Encoder(tf.keras.Model):
    def __init__(self, param):
        super(Encoder, self).__init__()
        self.layer_type = param.layer_type.lower()
        self.n_layers = param.encoder_layers
        self.units = param.units
        self.dropout = param.dropout
        self.embedding = tf.keras.layers.Embedding(
            param.encoder_vocab_size,
            param.embedding_dim,
            trainable=True
        )

        self.create_rnn_layers()

    def create_rnn_layers(self):
        self.rnn_layers = []
        for i in range(self.n_layers):
            # Only the last layer should not return sequences if we have multiple layers
            return_sequences = True if i < self.n_layers - 1 else False
            layer = get_layer(
                self.layer_type,
                self.units,
                self.dropout,
                return_sequences=return_sequences,
                return_state=True
            )
            self.rnn_layers.append(layer)

    def call(self, x, hidden):
        x = self.embedding(x)

        outputs = []
        states = []

        # Process the hidden state based on the number of layers
        if self.layer_type == "lstm":
            # For LSTM, we need h_state and c_state for each layer
            # hidden should be a list of [h1, c1, h2, c2, ..., hn, cn]
            hidden_states = []
            for i in range(self.n_layers):
                if 2*i+1 < len(hidden):
                    hidden_states.append([hidden[2*i], hidden[2*i+1]])
                else:
                    # Default if not enough states provided
                    hidden_states.append(None)
        else:
            # For GRU/RNN, we need one state per layer
            # hidden should be a list of [h1, h2, ..., hn]
            hidden_states = []
            for i in range(self.n_layers):
                if i < len(hidden):
                    hidden_states.append(hidden[i])
                else:
                    hidden_states.append(None)

        # Process through RNN layers
        for i, rnn in enumerate(self.rnn_layers):
            if i == 0:
                if hidden_states[i] is not None:
                    if self.layer_type == "lstm":
                        x, h_state, c_state = rnn(x, initial_state=hidden_states[i])
                        states.extend([h_state, c_state])
                    else:
                        x, state = rnn(x, initial_state=hidden_states[i])
                        states.append(state)
                else:
                    x, *state = rnn(x)
                    states.extend(state)
            else:
                if hidden_states[i] is not None:
                    if self.layer_type == "lstm":
                        x, h_state, c_state = rnn(x, initial_state=hidden_states[i])
                        states.extend([h_state, c_state])
                    else:
                        x, state = rnn(x, initial_state=hidden_states[i])
                        states.append(state)
                else:
                    x, *state = rnn(x)
                    states.extend(state)

            outputs.append(x)

        # The last output is what we want to return as encoder output
        return outputs[-1], states

    def initialize_hidden_state(self, batch_size):
        states = []
        if self.layer_type == "lstm":
            # For LSTM, we need h_state and c_state for each layer
            for _ in range(self.n_layers):
                h = tf.zeros((batch_size, self.units))
                c = tf.zeros((batch_size, self.units))
                states.extend([h, c])
        else:
            # For GRU/RNN, we need one state per layer
            for _ in range(self.n_layers):
                states.append(tf.zeros((batch_size, self.units)))

        return states

# **Decoder**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Assumed attention layer (you can replace it with your own class)
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, hidden, enc_output):
        # hidden: [batch_size, hidden_size]
        # enc_output: [batch_size, seq_len, hidden_size]
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

# Assumed helper function
def get_layer(layer_type, units, dropout, return_sequences, return_state):
    if layer_type.lower() == "lstm":
        return tf.keras.layers.LSTM(units,
                                    return_sequences=return_sequences,
                                    return_state=return_state,
                                    dropout=dropout)
    elif layer_type.lower() == "gru":
        return tf.keras.layers.GRU(units,
                                   return_sequences=return_sequences,
                                   return_state=return_state,
                                   dropout=dropout)
    elif layer_type.lower() == "rnn":
        return tf.keras.layers.SimpleRNN(units,
                                   return_sequences=return_sequences,
                                   return_state=return_state,
                                   dropout=dropout)
    else:
        raise ValueError(f"Unsupported RNN type: {layer_type}")

class Decoder(tf.keras.Model):
    def __init__(self, param):
        super(Decoder, self).__init__()

        self.layer_type = param.layer_type.lower()
        self.n_layers = param.decoder_layers
        self.units = param.units
        self.dropout = param.dropout
        self.attention = param.attention
        self.attention_type = getattr(param, 'attention_type', 'bahdanau')

        self.embedding_layer = layers.Embedding(
            input_dim=param.decoder_vocab_size,
            output_dim=param.embedding_dim,
            trainable=True
        )

        self.dense = layers.Dense(param.decoder_vocab_size, activation="softmax")

        if self.attention:
            self.attention_layer = BahdanauAttention(self.units)

        self.create_rnn_layers()

    def create_rnn_layers(self):
        self.rnn_layers = []
        for i in range(self.n_layers):
            return_sequences = True if i < self.n_layers - 1 else False
            layer = get_layer(
                self.layer_type, self.units, self.dropout,
                return_sequences=return_sequences, return_state=True
            )
            self.rnn_layers.append(layer)

    def call(self, x, hidden, enc_out=None):
        # x: [batch_size, 1]
        # hidden: list of states from encoder

        x = self.embedding_layer(x)  # [batch_size, 1, embedding_dim]

        attention_weights = None
        attention_vector = None

        # Format hidden states based on RNN type
        if self.layer_type == "lstm":
            # For LSTM, we need h_state and c_state for each layer
            # hidden should be a list of [h1, c1, h2, c2, ..., hn, cn]
            hidden_states = []
            for i in range(self.n_layers):
                if 2*i+1 < len(hidden):
                    hidden_states.append([hidden[2*i], hidden[2*i+1]])
                else:
                    # Default if not enough states provided
                    hidden_states.append(None)

            # Use first hidden state for attention
            attention_state = hidden[0] if hidden else None
        else:
            # For GRU/RNN, we need one state per layer
            hidden_states = []
            for i in range(self.n_layers):
                if i < len(hidden):
                    hidden_states.append(hidden[i])
                else:
                    hidden_states.append(None)

            # Use first hidden state for attention
            attention_state = hidden[0] if hidden else None

        # Apply attention if enabled
        if self.attention and enc_out is not None and attention_state is not None:
            context_vector, attention_weights = self.attention_layer(attention_state, enc_out)
            # Expand context vector to match x's time dimension
            context_vector = tf.expand_dims(context_vector, 1)
            # Concatenate context vector with input embedding
            x = tf.concat([context_vector, x], axis=-1)

        # Process through RNN layers
        states = []
        for i, rnn in enumerate(self.rnn_layers):
            if i == 0:
                if hidden_states[i] is not None:
                    if self.layer_type == "lstm":
                        x, h_state, c_state = rnn(x, initial_state=hidden_states[i])
                        states.extend([h_state, c_state])
                    else:
                        x, state = rnn(x, initial_state=hidden_states[i])
                        states.append(state)
                else:
                    x, *state = rnn(x)
                    states.extend(state)
            else:
                if hidden_states[i] is not None:
                    if self.layer_type == "lstm":
                        x, h_state, c_state = rnn(x, initial_state=hidden_states[i])
                        states.extend([h_state, c_state])
                    else:
                        x, state = rnn(x, initial_state=hidden_states[i])
                        states.append(state)
                else:
                    x, *state = rnn(x)
                    states.extend(state)

        # Apply the dense layer to get output probabilities
        # If the last RNN layer returns sequences, take only the last timestep
        if len(x.shape) > 2:
            x = x[:, -1, :]

        output = self.dense(x)

        return output, states, attention_weights


# **SequenceTOSequence**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.text import Tokenizer

from tqdm import tqdm
import random
import time
import wandb
import numpy as np
import pandas as pd
class SequenceTOSequence():
    def __init__(self, parameters):

        #Basic configurations
        self.param=parameters
        self.embedding_dim = parameters.embedding_dim
        self.encoder_layers = parameters.encoder_layers
        self.decoder_layers = parameters.decoder_layers
        self.layer_type = parameters.layer_type
        self.units = parameters.units
        self.dropout = parameters.dropout
        self.batch_size = parameters.batch_size

        #Add information regarding attention layer
        self.attention = parameters.attention
        self.attention_type = parameters.attention_type

        self.stats = []

        self.apply_beam_search = parameters.apply_beam_search

        #Early stop conditions
        self.patience=parameters.patience
        self.restoreBestModel=parameters.restoreBestModel

        #teacher forcing
        self.apply_teacher_forcing=parameters.apply_teacher_forcing
        self.teacher_forcing_ratio=parameters.teacher_forcing_ratio

    #Build model Add specific optimizers
    def build(self, loss, metric,optimizer='adam',lr=0.001):
        self.loss = loss

        #Select specific optimizer
        if(optimizer=='adam'):
          self.optimizer=tf.keras.optimizers.Adam(learning_rate=lr)
        if(optimizer=='nadam'):
          self.optimizer=tf.keras.optimizers.Nadam(learning_rate=lr)
        else:
          self.optimizer=tf.keras.optimizers.RMSprop(learning_rate=lr)

        self.metric = metric

    def set_vocabulary(self, input_tokenizer, targ_tokenizer):
        self.input_tokenizer = input_tokenizer
        self.targ_tokenizer = targ_tokenizer
        self.create_model()

    """This procedure used to define Encoder Decoder Layer"""
    def create_model(self):

        encoder_vocab_size = len(self.input_tokenizer.word_index) + 1
        decoder_vocab_size = len(self.targ_tokenizer.word_index) + 1
        self.param.encoder_vocab_size=encoder_vocab_size
        self.param.decoder_vocab_size=decoder_vocab_size
        #Add Encoder layer

        self.encoder = Encoder(self.param)

        #Create decode with or without any attention layer
        #Check following properties to add attention
        # self.attention
        # self.attention_type
        self.decoder = Decoder(self.param)


    @tf.function
    def train(self, input, target, enc_state):

        loss = 0

        with tf.GradientTape() as tape:

            enc_out, enc_state = self.encoder(input, enc_state)

            dec_state = enc_state
            dec_input = tf.expand_dims([self.targ_tokenizer.word_index["\t"]]*self.batch_size ,1)

            apply_teacher_forcing=False
            #decide whether to use teacher forcing
            if random.random() < self.teacher_forcing_ratio:
              apply_teacher_forcing=True
            ## We use Teacher forcing to train the network
            ## Each target at timestep t is passed as input for timestep t + 1
            if  (apply_teacher_forcing==True):
                #Apply teacher forcing
                for t in range(1, target.shape[1]):

                    preds, dec_state, _ = self.decoder(dec_input, dec_state, enc_out)
                    loss += self.loss(target[:,t], preds)
                    self.metric.update_state(target[:,t], preds)
                    #As teacher forcing  applied we pass next target as decoder input
                    dec_input = tf.expand_dims(target[:,t], 1)

            else:
                #Without teacher forcing

                for t in range(1, target.shape[1]):

                    preds, dec_state, _ = self.decoder(dec_input, dec_state, enc_out)
                    loss += self.loss(target[:,t], preds)
                    self.metric.update_state(target[:,t], preds)
                    #As teacher forcing not applied we pass decoder input as whatever we predict
                    preds = tf.argmax(preds, 1)
                    dec_input = tf.expand_dims(preds, 1)


            batch_loss = loss / target.shape[1]

            variables = self.encoder.variables + self.decoder.variables
            gradients = tape.gradient(loss, variables)

            self.optimizer.apply_gradients(zip(gradients, variables))

        return batch_loss, self.metric.result()




    def fit(self, dataset, val_dataset, batch_size=128, epochs=5, wandb=None,apply_teacher_forcing=True, teacher_forcing_ratio=0.7):

        self.batch_size = batch_size
        self.apply_teacher_forcing = apply_teacher_forcing
        self.teacher_forcing_ratio=teacher_forcing_ratio
        #Prepare chunk of data based on batch size provided
        steps_per_epoch = len(dataset) // self.batch_size
        #steps_per_epoch_val = len(val_dataset) // self.batch_size

        dataset = dataset.batch(self.batch_size, drop_remainder=False)
        #val_dataset = val_dataset.batch(self.batch_size, drop_remainder=False)


        sample_inp, sample_targ = next(iter(dataset))
        self.max_target_len = sample_targ.shape[1]
        self.max_input_len = sample_inp.shape[1]

        #Store Encoder ,decoder details in case model get good accuracy
        #Will be useful to restore best model
        self.bestEncoder=self.encoder
        self.bestDecoder=self.decoder
        self.bestoptimizer=self.optimizer

        accuracyDegradePatience=0
        self.oldaccuracy=0
        for epoch in  tqdm(range(1, epochs+1), total = epochs,desc="Epochs "):

            if(accuracyDegradePatience>=self.patience):
                if(self.restoreBestModel==True):
                    self.encoder=self.bestEncoder
                    self.decoder=self.bestDecoder
                    self.optimizer=self.bestoptimizer
                break
            ## Training loop ##
            total_loss = 0
            total_acc = 0
            self.metric.reset_state()

            starting_time = time.time()
            enc_state = self.encoder.initialize_hidden_state(self.batch_size)



            for batch, (input, target) in enumerate(dataset.take(steps_per_epoch)):
                #Accumulate loss and accurecy for each batch
                batch_loss, acc = self.train(input, target, enc_state)
                total_loss += batch_loss
                total_acc += acc
            #Calculate validation accurecy for current Epoch

            avg_acc = total_acc / steps_per_epoch
            avg_loss = total_loss / steps_per_epoch

            # Validation loop ##
            total_val_loss = 0
            total_val_acc = 0
            self.metric.reset_state()

            enc_state = self.encoder.initialize_hidden_state(self.batch_size)

            #Process data in batches

            avg_val_loss, avg_val_acc = self.evaluate(val_dataset,batch_size=self.batch_size)



            #Verify if model performance degrading.add()
            #In case train accurecy improved but no significant imprrovement in validation
            #Add condition for early stopping
            #Restore best model based on the input
            if(self.oldaccuracy>avg_val_acc):
              accuracyDegradePatience+=1
            else:
              self.bestEncoder=self.encoder
              self.bestDecoder=self.decoder
              self.bestoptimizer=self.optimizer
              self.oldaccuracy=avg_val_acc
              accuracyDegradePatience=0
            print( "\nTrain Loss: {0:.4f} Train Accuracy: {1:.4f} Validation Loss: {2:.4f} Validation Accuracy: {3:.4f}".format(avg_loss, avg_acc*100, avg_val_loss, avg_val_acc*100))

            time_taken = time.time() - starting_time

            #Add logs for WanDb
            self.stats.append({"epoch": epoch,
                            "train_loss": avg_loss,
                            "val_loss": avg_val_loss,
                            "train_acc": avg_acc*100,
                            "val_acc": avg_val_acc*100,
                            "training time": time_taken})

            #Log to wanDB
            if not (wandb is None):
                wandb.log(self.stats[-1])

            print(f"\nTime taken for the epoch {time_taken:.4f}")


        print("\nModel trained successfully !!")
    @tf.function
    def validation(self, inp, trgt, encoder_state):
        #Custom validation

        loss = 0
        #encoder input
        encoder_output, encoder_state = self.encoder(inp, encoder_state)

        #Set initial state of decoder from encoder state
        decoder_state = encoder_state
        decoder_input = tf.expand_dims([self.targ_tokenizer.word_index["\t"]]*self.batch_size ,1)

        for t in range(1, trgt.shape[1]):
            #Get decoder prediction
            prediction, decoder_state, _ = self.decoder(decoder_input, decoder_state, encoder_output)
            loss += self.loss(trgt[:,t], prediction)
            self.metric.update_state(trgt[:,t], prediction)

            prediction = tf.argmax(prediction, 1)
            decoder_input = tf.expand_dims(prediction, 1)

        batch_loss = loss / trgt.shape[1]

        return batch_loss, self.metric.result()
    def evaluate(self, test_dataset, batch_size=None):
        """Evaluate our model on test data"""
        if batch_size is not None:
            self.batch_size = batch_size

        #prepare chuck of data based on the batch size
        steps_per_epoch_test = len(test_dataset) // batch_size
        test_dataset = test_dataset.batch(batch_size, drop_remainder=True)

        total_test_loss = 0
        total_test_acc = 0
        self.metric.reset_state()

        enc_state = self.encoder.initialize_hidden_state(self.batch_size)

        #print("\nRunning test dataset through the model...\n")
        #Run in batches based on the input batch size
        for batch, (input, target) in enumerate(test_dataset.take(steps_per_epoch_test)):
            batch_loss, acc = self.validation(input, target, enc_state)
            total_test_loss += batch_loss
            total_test_acc += acc

        #Caculate avarage  test accuracy and loss
        avg_test_acc = total_test_acc / steps_per_epoch_test
        avg_test_loss = total_test_loss / steps_per_epoch_test

        #Display details
        #print(f"Test Loss: {avg_test_loss:.4f} Test Accuracy: {avg_test_acc:.4f}")

        return avg_test_loss, avg_test_acc

    """ This function used to translate english world to respective language"""

    def translate(self, word, get_heatmap=False):
        #start and end token for input word
        start="\t"
        end="\n"
        word =start  + word + end

        #Tokenize input and perform  preprocessing
        inputs = self.input_tokenizer.texts_to_sequences([word])
        inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                               maxlen=self.max_input_len,
                                                               padding="post")

        result = ""
        att_wts = []

        #Process input through encoder
        enc_state = self.encoder.initialize_hidden_state(1)
        enc_out, enc_state = self.encoder(inputs, enc_state)

        # Set initial decoder sate to encoder state
        dec_state = enc_state
        dec_input = tf.expand_dims([self.targ_tokenizer.word_index[start]]*1, 1)

        #Run the loop for maximum word size the target language can have
        #We get this data during training
        for t in range(1, self.max_target_len):

            preds, dec_state, attention_weights = self.decoder(dec_input, dec_state, enc_out)

            #Add attention weights which is useful for generating attention heatmaps
            if get_heatmap:
                att_wts.append(attention_weights)

            #Pass the current prediction as input to next iteration
            preds = tf.argmax(preds, 1)

            #Accumulate target words
            next_char = self.targ_tokenizer.index_word[preds.numpy().item()]
            result += next_char

            #Decoder input for next iteration
            dec_input = tf.expand_dims(preds, 1)

            #If we receive end token stop the loop
            if next_char == end:
                break

        return result[:-1], att_wts[:-1]



In [ ]:

from tensorflow.keras import layers
from tqdm import tqdm
def get_layer(name, units, dropout, return_state=False, return_sequences=False):

    if name=="rnn":
        return layers.SimpleRNN(units=units, dropout=dropout,
                                return_state=return_state,
                                return_sequences=return_sequences)

    if name=="gru":
        return layers.GRU(units=units, dropout=dropout,
                          return_state=return_state,
                          return_sequences=return_sequences)

    if name=="lstm":
        return layers.LSTM(units=units, dropout=dropout,
                           return_state=return_state,
                           return_sequences=return_sequences)


In [ ]:
import wandb

In [ ]:
wandb.init()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: na21b050 (na21b050-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


# Data Preprocessing

In [ ]:

START_TOKEN="\t"
END_TOKEN="\n"


In [ ]:
downloadDataSet()

downloading....
download Complete
Extracting..
Complete
You are all set


In [ ]:
language="te"
train_dir, val_dir, test_dir = get_files(language)



In [ ]:

dataset, input_tokenizer, targ_tokenizer = preprocess_data(train_dir)
val_dataset, _, _ = preprocess_data(val_dir,input_tokenizer,targ_tokenizer)

In [ ]:
#train data
dataset, input_tokenizer, targ_tokenizer = preprocess_data(train_dir)

## Sweep Configuration

In [ ]:
sweep_config = {
  "name": "DL_Assignment3_Rnn",
  "method": "bayes",
  "metric": {
      "name": "val_acc",
      "goal": "maximize",
  },

  "parameters": {
        "num_of_encoders":{
          "values":[1,2,3,4]
        },
        "num_of_decoders":{
            "values":[1,2,3]

        },
        "cell_type":{
          "values":['lstm','gru','rnn']
        },

        "lr":{
          "values":[0.05,0.001,0.005]
        },
        "optimizer":{
          "values":['adam','rmsprop']
        },
        "dropout":{ "values": [0.3,0.5]},
        "latent_dim":{ "values": [64,128,256,512]},
        "inp_emb_size": {"values": [64,128,256]},
        "batch_size":{"values":[256,128,64]},

        }
    }

In [ ]:
# This is the main function to use to train/fine-tune the model using wandb runs
def train_wandb():
    run = wandb.init()

    config=wandb.config
    # Set the run name
    name="num_of_encoders("+ str(config["num_of_encoders"]) + ")_"
    name = " num_of_decoders(" + str(config["num_of_decoders"]) + ")_"
    name += " cell_type(" + str(config["cell_type"]) + ")_"

    name += "latent_dim(" + str(config["latent_dim"])+ ")_"
    name += "lr(" + str(config["lr"])+ ")_"
    name += "optimizer(" + str(config["optimizer"]) + ")_"
    name += "dropout(" + str(config["dropout"]) + ")"
    name += "inp_emb_size(" + str(config["inp_emb_size"]) + ")_"
    name+="batch_size(" + str(config["batch_size"]) + ")"


    wandb.run.name = name[:-1]
    batch_size=config["batch_size"]
    inp_emb_size=config["inp_emb_size"]
    dropout=config["dropout"]
    optimizer=config["optimizer"]
    num_of_encoders=config["num_of_encoders"]
    num_of_decoders=config["num_of_decoders"]

    lr=config["lr"]
    latent_dim=config["latent_dim"]
    cell_type=config["cell_type"]


    param=Parameters(language="te",\
                        embedding_dim=inp_emb_size,\
                        encoder_layers=num_of_encoders,\
                        decoder_layers=num_of_decoders,\
                        layer_type=cell_type,\
                        units=latent_dim,\
                        dropout=dropout,
                        epochs=15,\
                 batch_size=batch_size\
                   )
    param.apply_teacher_forcing=True
    param.teacher_forcing_ratio=1
    param.patience=5
    model = SequenceTOSequence(param)
    model.set_vocabulary(input_tokenizer, targ_tokenizer)

    model.build(loss=tf.keras.losses.SparseCategoricalCrossentropy(),\
                metric = tf.keras.metrics.SparseCategoricalAccuracy(),\
                optimizer = optimizer,\
                lr=lr\
                )

    model.fit(dataset, val_dataset, epochs=param.epochs, wandb=wandb,teacher_forcing_ratio=param.teacher_forcing_ratio)



In [ ]:


sweep_id = wandb.sweep(sweep_config,entity="na21b050-iit-madras", project="CS6910_Assignment3")


Create sweep with ID: h16aspho
Sweep URL: https://wandb.ai/na21b050-iit-madras/CS6910_Assignment3/sweeps/h16aspho


# **Sweep Info**

In [ ]:

wandb.agent(sweep_id, train_wandb)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: y6ofsyiy with config:
wandb: 	batch_size: 128
wandb: 	cell_type: rnn
wandb: 	dropout: 0.3
wandb: 	inp_emb_size: 256
wandb: 	latent_dim: 256
wandb: 	lr: 0.05
wandb: 	num_of_decoders: 2
wandb: 	num_of_encoders: 3
wandb: 	optimizer: rmsprop


Epochs :   0%|          | 0/15 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:774: UserWarning: Gradients do not exist for variables ['seed_generator/seed_generator_state', 'seed_generator_1/seed_generator_state', 'encoder/simple_rnn_2/simple_rnn_cell/kernel', 'encoder/simple_rnn_2/simple_rnn_cell/recurrent_kernel', 'encoder/simple_rnn_2/simple_rnn_cell/bias', 'seed_generator_2/seed_generator_state', 'seed_generator_3/seed_generator_state', 'seed_generator_4/seed_generator_state'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(
Epochs :   7%|▋         | 1/15 [01:26<20:05, 86.11s/it]


Train Loss: 3.4254 Train Accuracy: 57.7977 Validation Loss: 10.7165 Validation Accuracy: 59.0177

Time taken for the epoch 86.1093


Epochs :  13%|█▎        | 2/15 [02:14<13:48, 63.76s/it]


Train Loss: 2.7535 Train Accuracy: 63.4134 Validation Loss: 45.0075 Validation Accuracy: 1.6505

Time taken for the epoch 48.1003


...


Train Loss: 0.5015 Train Accuracy: 83.9845 Validation Loss: 4.2722 Validation Accuracy: 48.9412

Time taken for the epoch 62.7168


Epochs :  20%|██        | 3/15 [03:54<14:38, 73.18s/it]


Train Loss: 0.4096 Train Accuracy: 86.8774 Validation Loss: 4.5404 Validation Accuracy: 49.1166

Time taken for the epoch 62.7030


Epochs :  27%|██▋       | 4/15 [04:58<12:41, 69.25s/it]


Train Loss: 0.3543 Train Accuracy: 88.7435 Validation Loss: 4.6818 Validation Accuracy: 49.7600

Time taken for the epoch 63.2156
